In [2]:
import pandas as pd
import numpy as np
import os
import os.path, time
import zipfile as zp
import re
import random
from datetime import datetime, date
from datetime import datetime, timedelta
from scipy import optimize
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
import unicodedata
#import seaborn as sns; sns.set()

### Uncertainty, Risk (DOSPERT, IUS, LOT-R)

In [3]:
def dospert_domain_label(subID):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
    
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    df = pd.read_csv(subfile)
    S = [[10,'01'],[10,'08'],[20,'03'],[20,'06'],[30,'07'],[30,'10'],[40,'07'],[40,'08']]
    R = [[10,'02'],[10,'04'],[20,'02'],[20,'04'],[20,'07'],[30,'04'],[40,'02'],[40,'03']]
    F = [[10,'03'],[10,'05'],[10,'09'],[20,'05'],[20,'08'],[20,'09'],[30,'03'],[30,'06']]
    H = [[10,'06'],[20,'10'],[30,'02'],[30,'05'],[40,'01'],[40,'04'],[40,'05'],[40,'06']]
    E = [[10,'07'],[10,'10'],[20,'01'],[30,'01'],[30,'08'],[30,'09'],[40,'09'],[40,'10']]
    survey = ['DOSPERT-10', 'DOSPERT-20','DOSPERT-30','DOSPERT-40']
    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([subID])]
    ser_title = subsetDataFrame['survey_title'].to_list()
    q_title =  subsetDataFrame['question_title'].to_list()
    n = [] #question_title
    for s in range(len(q_title)):
        if q_title[s][1] == '.':
            n.append('0'+q_title[s][0])
        else:
            n.append(q_title[s][0:2])
    #print(n)
    answer_title = subsetDataFrame['answer_value'].to_list()
    zip_title = []
    domain_label = []
    answer =[]
    for i in range(len(ser_title)):
        zip_title.append([int(ser_title[i][-2:]),n[i]])
        #print(zip_title[i])
        answer.append(int(answer_title[i][0]))
        if zip_title[i] in S:
            domain_label.append('S')
        if zip_title[i] in R:
            domain_label.append('R')
        if zip_title[i] in F:
            domain_label.append('F')
        if zip_title[i] in H:
            domain_label.append('H')
        if zip_title[i] in E:
            domain_label.append('E')
    return domain_label, answer

def dosert(subID):

    domain_label, answer = dospert_domain_label(subID)

    dospert = pd.DataFrame([domain_label,answer ]).T 
    dospert.columns = ["domain_label", "answer"]

    dospert_domain = dospert.groupby(dospert.domain_label).sum()
    dosert_score = dospert_domain.answer.to_list()
    domain = dospert_domain.index.to_list()
    return domain, dosert_score
#40 is the highest likelihood of each domain

def IUS_factor_label(subID):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
    
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    
    df = pd.read_csv(subfile)
    f1 =[[9,1], [9,2], [9,3], [9,9], [8,3], [8,4], [8,5], [8,6], [8,7], [8,8], [7,2], [7,4], [7,5], [7,6],[7,7]]
    f2 = [[9,4], [9,5], [9,6], [9,7], [9,8], [8,1], [8,2], [8,9], [7,1], [7,3], [7,8], [7,9]]
    survey = ['IUS-9', 'IUS-18','IUS-27']
    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([subID])]
    ser_title = subsetDataFrame['survey_title'].to_list()
    q_title =  subsetDataFrame['question_title'].to_list()
    answer_title = subsetDataFrame['answer_value'].to_list()
    zip_title = []
    domain_label = []
    answer =[]
    for i in range(len(ser_title)):
        zip_title.append([int(ser_title[i][-1]),int(q_title[i][0])])
        answer.append(int(answer_title[i][0]))
        if zip_title[i] in f1:
            domain_label.append('f1')
        if zip_title[i] in f2:
            domain_label.append('f2')
       
    return domain_label, answer

def IUS_score(subID):
    
    domain_label, answer = IUS_factor_label(subID)
    ius = pd.DataFrame([domain_label,answer ]).T 
    ius.columns = ["domain_label", "answer"]

    ius_domain = ius.groupby(ius.domain_label).sum()
    ius_score = ius_domain.answer.to_list()
    domain = ius_domain.index.to_list()
    return domain, ius_score

def lot(subID):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
    
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    df = pd.read_csv(subfile)
    opt_des = ["In uncertain times, I usually expect the best.","I'm always optimistic about my future.",\
           "Overall, I expect more good things to happen to me than bad."]
    pes_des = ["If something can go wrong for me, it will.","I hardly ever expect things to go my way.",\
          "I rarely count on good things happening to me."]
    df_temp = df[df['survey_title'].isin(['LOT-R']) & df['participant_id'].isin([subID])]
    question = df_temp['question_title'].to_list()
    answer = df_temp['answer_value'].to_list()
    opt_score = 0
    pes_score = 0
    for i in range(len(question)):
        if question[i] in opt_des:
        #print(answer[i])
            if answer[i][0] == 'A':
                opt_score+= 5
            if answer[i][0] == 'B':
                opt_score+= 4
            if answer[i][0] == 'C':
                opt_score+= 3
            if answer[i][0] == 'D':
                opt_score+= 2
            if answer[i][0] == 'E':
                opt_score+= 1
        if question[i] in pes_des:
        #print(answer[i])
            if answer[i][0] == 'A':
                pes_score+= 5
            if answer[i][0] == 'B':
                pes_score+= 4
            if answer[i][0] == 'C':
                pes_score+= 3
            if answer[i][0] == 'D':
                pes_score+= 2
            if answer[i][0] == 'E':
                pes_score+= 1
    
    return ['opt','pes'], [opt_score,pes_score]

### Impulsivity

In [4]:
def bisbas_label(subID):
    BAS =[[12,'03'], [12,'09'], [12,'12'], [24,'09'], [12,'05'], [12,'10'], [24,'03'], [24,'08'], \
      [12,'04'], [12,'07'], [24,'02'], [24,'06'], [12,'11']]
    BIS = [[12,'02'], [12,'08'], [24,'01'], [24,'04'], [24,'07'], [24,'10'], [24,'12']]
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
    
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    df = pd.read_csv(subfile)    
    survey = ['BIS/BAS-12', 'BIS/BAS-24']
    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([subID])]
    ser_title = subsetDataFrame['survey_title'].to_list()
    q_title =  subsetDataFrame['question_title'].to_list()
    n = [] #question_title
    for s in range(len(q_title)):
        if q_title[s][1] == '.':
            n.append('0'+q_title[s][0])
        else:
            n.append(q_title[s][0:2])
            
    answer_value = subsetDataFrame['answer_value'].to_list()
    #print(answer_value)
    zip_title = []
    domain_label = []
    answer =[]
    for i in range(len(ser_title)):
        zip_title.append([int(ser_title[i][-2:]),n[i]])
        #print(zip_title[i])
        if zip_title[i] == [12,'02']:
            #print(i)
            #print(answer_value[i][0])
            if int(answer_value[i][0]) == 1:
                answer.append(4)
            if int(answer_value[i][0]) == 2:
                answer.append(3)
            if int(answer_value[i][0]) == 3:
                answer.append(2)
            if int(answer_value[i][0]) == 4:
                answer.append(1)
        elif zip_title[i] == [12,'08']:
            #print(i)
            #print(answer_value[i][0])
            if int(answer_value[i][0]) == 1:
                answer.append(4)
            if int(answer_value[i][0]) == 2:
                answer.append(3)
            if int(answer_value[i][0]) == 3:
                answer.append(2)
            if int(answer_value[i][0]) == 4:
                answer.append(1)
        else:
            answer.append(int(answer_value[i][0]))
            #print(answer)
        if zip_title[i] in BAS:
            domain_label.append('bas')
        if zip_title[i] in BIS:
            domain_label.append('bis')
        
    return domain_label, answer

def bisbas(subID):

    domain_label, answer = bisbas_label(subID)

    bisbas = pd.DataFrame([domain_label,answer ]).T 
    bisbas.columns = ["domain_label", "answer"]

    bisbas_domain = bisbas.groupby(bisbas.domain_label).sum()
    bisbas_score = bisbas_domain.answer.to_list()
    domain = bisbas_domain.index.to_list()
    return domain, bisbas_score

def UPPS(subID):
    #non_reverse =[[12,'01'],[12,'04'],[12,'06'],[12,'11'],[24,'02'],[24,'04'],[24,'07'],[24,'09'],[24,'12'],[36,'03'],\
              #[36,'04'],[36,'08'],[36,'09'],[48,'01'],[48,'02'],[48,'06'],[48,'07'],[48,'12'],[59,'05'],[59,'07']]
    reverse = [[12,'02'],[12,'03'],[12,'05'],[12,'07'],[12,'08'],[12,'09'],[12,'10'],[12,'12'],
          [24,'01'],[24,'03'],[24,'05'],[24,'06'],[24,'08'],[24,'10'],[24,'11'],
          [36,'01'],[36,'02'],[36,'05'],[36,'06'],[36,'07'],[36,'10'],[36,'11'],[36,'12'],
          [48,'03'],[48,'04'],[48,'05'],[48,'08'],[48,'09'],[48,'10'],[48,'11'],
          [59,'01'],[59,'02'],[59,'03'],[59,'04'],[59,'06'],[59,'07'],[59,'08'],[59,'09'],[59,'10'],[59,'11']]
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
    
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv' 
    df = pd.read_csv(subfile)
    survey = ['UPPS-36', 'UPPS-48', 'UPPS-59', 'UPPS-24', 'UPPS-12']
    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([subID])]
    survey_title = subsetDataFrame['survey_title'].to_list()
    question_title =  subsetDataFrame['question_title'].to_list()
    n = [] #question_title
    for s in range(len(question_title)):
        if question_title[s][1] == '.':
            n.append('0'+question_title[s][0])
        else:
            n.append(question_title[s][0:2])
    #print(n)
    answer_value = subsetDataFrame['answer_value'].to_list()
    answer =[]
    for i in range(len(survey_title)):
    
        zip_title = [int(survey_title[i][-2:]),n[i]]
        if zip_title in reverse:
        
            if int(answer_value[i][0]) == 1:
                answer.append(4)
            
            if int(answer_value[i][0]) == 2:
                answer.append(3)
            
            if int(answer_value[i][0]) == 3:
                answer.append(2)
            
            if int(answer_value[i][0]) == 4:
                answer.append(1)       
        else:
            answer.append(int(answer_value[i][0]))
    
    result = sum(answer)
    return result

def bis11(subID):

    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv' 
    df = pd.read_csv(subfile)
    survey = ['BIS11-15', 'BIS11-30']
    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([subID])]
    question_title = subsetDataFrame['question_title'].to_list()
    answer_value = subsetDataFrame['answer_value'].to_list()
    reverse = ['12. I am a careful thinker.','1. I plan tasks carefully.','7. I plan trips well ahead of time.',
              'I save regularly.','8.  I am self controlled.','15. I like to think about complex problems.',
               '13. I plan for job security.','I concentrate easily.','15.  I am future oriented.',
               '5. I am a steady thinker.','14. I like puzzles.','8. I can only think about one thing at a time.']
    answer = []
    for i in range(len(question_title)):
        if  question_title[i] in reverse:

            if int(answer_value[i][0]) == 1:
                answer.append(4)

            if int(answer_value[i][0]) == 2:
                answer.append(3)

            if int(answer_value[i][0]) == 3:
                answer.append(2)

            if int(answer_value[i][0]) == 4:
                answer.append(1)       
        else:
            answer.append(int(answer_value[i][0]))
    return sum(answer)

### Anxiety

In [5]:
def phq(subID):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
    
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    df = pd.read_csv(subfile)
    survey = ['PHQ8']
    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([subID])]

    answer_value = subsetDataFrame['answer_value'].to_list()
    answer = []
    for i in range(len(answer_value)):
        answer.append(int(answer_value[i][0]))
    #print(answer)
    return sum(answer)

def stai_s(subID):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
    
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    df = pd.read_csv(subfile)
    reverse = ['I feel calm.', 'I feel secure.', 'I feel at ease.','I feel satisfied.','I feel self-confident.','I am relaxed.'
          'I feel content.','I feel steady','I feel pleasant.']
    survey = ['STAI-State']
    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([subID])]

    question_title =  subsetDataFrame['question_title'].to_list()
    answer_value = subsetDataFrame['answer_value'].to_list()

    answer =[]
    for i in range(len(question_title)):

        if question_title[i] in reverse:
        
            if int(answer_value[i][0]) == 1:
                answer.append(4)
            
            if int(answer_value[i][0]) == 2:
                answer.append(3)

            if int(answer_value[i][0]) == 3:
                answer.append(2)

            if int(answer_value[i][0]) == 4:
                answer.append(1)       
        else:
            answer.append(int(answer_value[i][0]))

    result = sum(answer)
    return result

def stai_t(subID):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    df = pd.read_csv(subfile)
    survey = ['STAI-Trait']
    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([subID])]
    reverse = ['I feel pleasant','I feel satisfied with myself','I feel rested','I am "calm, cool, and collected"',
               'I am happy','I feel secure','I make decisions easily','I am content','I am a steady person']
    question_title =  subsetDataFrame['question_title'].to_list()
    answer_value = subsetDataFrame['answer_value'].to_list()

    answer =[]
    for i in range(len(question_title)):

        if question_title[i] in reverse:

            if int(answer_value[i][0]) == 1:
                answer.append(4)

            if int(answer_value[i][0]) == 2:
                answer.append(3)

            if int(answer_value[i][0]) == 3:
                answer.append(2)

            if int(answer_value[i][0]) == 4:
                answer.append(1)       
        else:
            answer.append(int(answer_value[i][0]))

    result = sum(answer)
    return result

def BSI(subID):

    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    df = pd.read_csv(subfile)

    somatization = ['Faintness or dizziness','Pains in the heart or chest','Nausea or upset stomach',
            'Trouble getting your breath','Hot or cold spells.','Numbness or tingling in parts of your body',
           'Feeling weak in parts of your body']
    obssession = ['Trouble remembering things','Feeling blocked in getting things done',
               'Having to check and double check what you do','Difficulty making decisions','Your mind going blank',
              'Trouble concertrating']
    interpersonal = ['Your feelings being easily hurt','Feeling that people are unfriendly or dislike you',
                    'Feeling inferior to others']

    anxiety = ['Nervousness or shakiness inside','Suddenly scared for no reason','Feeling fearful',
               'Feeling tense or keyed up','Spells of terror or panic','Feeling so restless you couldn’t sit still']

    hosbility = ['Feeling easily annoyed or irritated','Temper outbursts that you could not control',
                'Having urges to beat, injure, or harm someone','Having urges to break or smash things',
                'Getting into frequent arguments']

    phobic = ['Feeling afraid in open spaces','Feeling afraid to travel on buses, subways, or trains',
             'Having to avoid certain things, places, or activities because they frighten you','Feeling uneasy in crowds',
             'Feeling nervous when you are left alone']

    paranoid = ['Feeling others are to blame for most of your troubles','Feeling that most people cannot be trusted',
               'Feeling that you are watched or talked about by others',
                'Others not giving you proper credit for your achievements','Feeling that people will take advantage of you if you let them']

    psychoticism = ['The idea that someone else can control your thoughts','Feeling lonely even when you are with people',
                   'The idea that you should be punished for your sins','Never feeling close to another person',
                   'The idea that something is wrong with your mind']
    appetite = ['Poor appetite']
    sleep = ['Trouble falling asleep']
    #death = ['Thoughts of death or dying']
    guilt = ['Feeling of guilt']

    survey = ['BSI-9', 'BSI-20', 'BSI-36', 'BSI-45', 'BSI-53']
    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([subID])]
    survey_title = subsetDataFrame['survey_title'].to_list()
    question_title =  subsetDataFrame['question_title'].to_list()
    answer_value = subsetDataFrame['answer_value'].to_list()
    for i in range(len(answer_value)):
        if answer_value[i] == 'Refused':
            answer_value[i] = '0'
    answer =[]
    question = []
    for i in range(len(question_title)):
        new_str = unicodedata.normalize("NFKD", question_title[i])
        question.append(new_str.strip())
    som = []
    obs = []
    inte = []
    anx = []
    hos = []
    pho = []
    par = []
    psy = []
    ape = []
    slp = []
    #dea = []
    gui = []

    for i in range(len(question_title)):
        if question[i][3:] in somatization:
            som.append(int(answer_value[i][0]))
        if question[i][4:] in somatization:
            som.append(int(answer_value[i][0]))

        if question[i][3:] in obssession:
            obs.append(int(answer_value[i][0]))
        if question[i][4:] in obssession:
            obs.append(int(answer_value[i][0]))

        if question[i][3:] in interpersonal:
            inte.append(int(answer_value[i][0]))
        if question[i][4:] in interpersonal:
            inte.append(int(answer_value[i][0]))

        if question[i][3:] in anxiety:
            anx.append(int(answer_value[i][0]))
        if question[i][4:] in anxiety:
            anx.append(int(answer_value[i][0]))

        if question[i][3:] in hosbility:
            hos.append(int(answer_value[i][0]))
        if question[i][4:] in hosbility:
            hos.append(int(answer_value[i][0]))

        if question[i][3:] in phobic:
            pho.append(int(answer_value[i][0]))
        if question[i][4:] in phobic:
            pho.append(int(answer_value[i][0]))

        if question[i][3:] in paranoid:
            par.append(int(answer_value[i][0]))
        if question[i][4:] in paranoid:
            par.append(int(answer_value[i][0]))

        if question[i][3:] in psychoticism:
            psy.append(int(answer_value[i][0]))
        if question[i][4:] in psychoticism:
            psy.append(int(answer_value[i][0]))
            
        if question[i][3:] in appetite:
            ape.append(int(answer_value[i][0]))
        if question[i][4:] in appetite:
            ape.append(int(answer_value[i][0]))
            
        if question[i][3:] in sleep:
            slp.append(int(answer_value[i][0]))
        if question[i][4:] in sleep:
            slp.append(int(answer_value[i][0])) 
            
        if question[i][3:] in guilt:
            gui.append(int(answer_value[i][0]))
        if question[i][4:] in guilt:
            gui.append(int(answer_value[i][0]))  
            
            
    psycharity = [sum(som), sum(obs), sum(inte), sum(anx), sum(hos), sum(pho), sum(par), sum(psy), 
                  sum(ape), sum(slp), sum(gui)]
    return psycharity

def BAI(subID):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    if subID in subID99:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    df = pd.read_csv(subfile)
    survey = ['BAI']
    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([subID])]
    
    question_title =  subsetDataFrame['question_title'].to_list()
    answer_value = subsetDataFrame['answer_value'].to_list()

    answer =[]
    for i in range(len(question_title)):

        
        answer.append(int(answer_value[i][0]))
        

    result = sum(answer)
    return result

### cognitvie task

In [6]:
def Raven_score(subID):
    sub = int(subID)
    Raven= pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Cognition/Raven.csv").set_index("subID").T.reset_index()
    Answer = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Cognition/Raven_Answer.txt", sep=",", header=None)
    Answer = Answer.T
    Answer.columns = ["Answer"]
    Raven["Answer"] = Answer
    Raven['Answer_copy'] = Raven['Answer']
    Raven1 = Raven.apply(lambda x: np.where(x == Raven["Answer_copy"], 1, 0), axis = 0).reset_index().drop(["Answer_copy"], axis = 1)
    final = Raven1.drop(columns = ["level_0", "index","Answer"])
    namelist = final.columns
    if sub in namelist:
        iq = final[sub].sum()
        if np.shape(iq) != ():
            iq = max(iq)
    else:
        iq = float("NaN")
    
    return iq


def numeracy(subID):
    Numeracy = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Cognition/Numeracy.csv")
    values = [0, 1]
    
    Numeracy["Q1"][1: ] = Numeracy["Q1"][1: ].apply(lambda x: re.sub(r'[a-z$,]', '', str(x)))
    correctQ1 = [(Numeracy["Q1"] != "0.05"),(Numeracy["Q1"] == "0.05")]
    Numeracy["Correct%Q1"] = np.select(correctQ1, values)

    Numeracy["Q2"][1: ] = Numeracy["Q2"][1: ].apply(lambda x: re.sub(r'[a-z,]', '', str(x)))
    correctQ2 = [(Numeracy["Q2"] != "5"),(Numeracy["Q2"] == "5")]
    Numeracy["Correct%Q2"] = np.select(correctQ2, values)

    Numeracy["Q3"][1: ] = Numeracy["Q3"][1: ].apply(lambda x: re.sub(r'[a-z,]', '', str(x)))
    correctQ3 = [(Numeracy["Q3"] != "47"), (Numeracy["Q3"] == "47")]
    Numeracy["Correct%Q3"] = np.select(correctQ3, values)

    Numeracy["Q4"][1: ] = Numeracy["Q4"][1: ].apply(lambda x: re.sub(r'[a-z$,]', '', str(x)))
    correctQ4 = [(Numeracy["Q4"] != "150"),(Numeracy["Q4"] == "150")]
    Numeracy["Correct%Q4"] = np.select(correctQ4, values)

    Numeracy["Q5"][1: ] = Numeracy["Q5"][1: ].apply(lambda x: re.sub(r'[a-z,%]', '', str(x)))
    correctQ5 = [(Numeracy["Q5"] != "1"),(Numeracy["Q5"] == "1")]
    Numeracy["Correct%Q5"] = np.select(correctQ5, values)
    
    Numeracy["Q6"][1: ] = Numeracy["Q6"][1: ].apply(lambda x: re.sub(r'[a-z,%]', '', str(x)))
    correctQ6 = [(Numeracy["Q6"] != "2. Ten percent (10%)"), (Numeracy["Q6"] == "2. Ten percent (10%)")]
    Numeracy["Correct%Q6"] = np.select(correctQ6, values)
    
    Numeracy["Q7"][1: ] = Numeracy["Q7"][1: ].apply(lambda x: re.sub(r'[a-z,%]', '', str(x)))
    correctQ7 = [(Numeracy["Q7"] != "3. Something that happens 1 in 10 times"),
            (Numeracy["Q7"] == "3. Something that happens 1 in 10 times")]
    Numeracy["Correct%Q7"] = np.select(correctQ7, values)

    Numeracy["Q8"][1: ] = Numeracy["Q8"][1: ].apply(lambda x: re.sub(r'[a-z$,]', '', str(x)))
    correctQ8 = [(Numeracy["Q8"] == "400000"), 
                 (Numeracy["Q8"] != "400000")]
    Numeracy["Correct%Q8"] = np.select(correctQ8, values)

    Numeracy["Q9"][1: ] = Numeracy["Q9"][1: ].apply(lambda x: re.sub(r'[a-z$,]', '', str(x)))
    correctQ9 = [(Numeracy["Q9"] == "242"), (Numeracy["Q9"] != "242")]
    Numeracy["Correct%Q9"] = np.select(correctQ9, values)
    
    Numeracy["NumeracyFinal"] = Numeracy["Correct%Q1"] + Numeracy["Correct%Q2"] + Numeracy["Correct%Q3"] + Numeracy["Correct%Q4"] + Numeracy["Correct%Q5"] + Numeracy["Correct%Q6"] + Numeracy["Correct%Q7"] + Numeracy["Correct%Q8"] + Numeracy["Correct%Q9"]
    df = Numeracy[['Q15', 'NumeracyFinal']][1:]
    sublist = Numeracy['Q15'][1:].tolist()
    if subID in sublist:
        score = df['NumeracyFinal'][df['Q15'] == subID].tolist()[0]
    else:
        score = float('NaN')
    return score

### RISK  - alpha, beta, gamma, ratio, RT

In [7]:
def rp_ratio(week,sub):
    df = raw_data(week,sub)

    
    amb = df.query('data_rounds_trial_type == "ambiguity"')
    amb_d = amb[['data_rounds_reward_value', 'data_rounds_selected_sure_bet_reward']]
    amb_ratio = 1 - (round(amb_d.data_rounds_selected_sure_bet_reward.sum()/36,2))
  
    
    risk = df.query('data_rounds_trial_type == "risk"')
    risk_d = risk[['data_rounds_reward_value', 'data_rounds_selected_sure_bet_reward']]
    risk_ratio = 1 - (round(risk_d.data_rounds_selected_sure_bet_reward.sum()/34,2))
    
    return amb_ratio, risk_ratio

def subID_list():
    subID = ['797','806','809','826','843','845','855','856','857','1031','1153','1154','1157','1174','1262','1264','1266',
        '1364','1367','1379','1380','1381','1384','1435','1465','1467','1468','1489','1492','1493','1494','1495','1498','1519',
        '1520','1527','1532','1536','1541','1721','1722','1737','1741','1788','1791','1810','1812','1833','1934']
    return subID
    

def raw_data(week,sub):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    subID = subID_list()
    
    week_label = [['01','01'],['08','11'],['17','19'],['23','26'],['32','34'],['36','42'],['44','48'],['52','56']]
    if (sub in subID99) == True:
        lottery_file_1 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_99_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][0]
        filelist_1 = zp.ZipFile(lottery_file_1).namelist()
        lottery_file_2 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_99_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][1]
        filelist_2 = zp.ZipFile(lottery_file_2).namelist()
    else:
        lottery_file_1 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_114_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][0]
        filelist_1 = zp.ZipFile(lottery_file_1).namelist()
        lottery_file_2 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_114_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][1]
        filelist_2 = zp.ZipFile(lottery_file_2).namelist()
        
    sub_file = {}
    for i in range(len(subID)):
        key = subID[i]
        sub_file.setdefault(key, [])
        sub_file[key].extend([s for s in filelist_1 if subID[i] in s])
        sub_file[key].extend([s for s in filelist_2 if subID[i] in s])

    df1 = pd.read_csv(zp.ZipFile(lottery_file_1).open(sub_file[sub][0]))
    df2 = pd.read_csv(zp.ZipFile(lottery_file_2).open(sub_file[sub][1]))
    df = pd.concat([df1, df2])
    #RT_start = df['start_time']
    #RT_end = df['end_time']
    return df

def analysis_data(df):
    df.data_rounds_selected_sure_bet_reward = df.data_rounds_selected_sure_bet_reward.astype(int)
    df_temp = df[['participant_id','data_rounds_reward_probability','data_rounds_reward_value','data_rounds_selected_sure_bet_reward','data_rounds_trial_type','data_rounds_ambiguity']]
    df_temp.loc[df.data_rounds_trial_type  == 'ambiguity', 'data_rounds_reward_probability'] = 0.5
    df_temp.loc[:,'data_rounds_selected_sure_bet_reward'] = df_temp['data_rounds_selected_sure_bet_reward'].replace({0:1, 1:0})
    df_temp = df_temp.rename(columns={"data_rounds_selected_sure_bet_reward": "data_rounds_selected_variable_bet_reward"})
    x = np.array(df_temp['data_rounds_reward_value'])
    x = np.reshape(x,[70,1])
    p = np.array(df_temp['data_rounds_reward_probability'])
    p = np.reshape(p,[70,1])
    s = 5*np.ones((70,1))
    A = np.array(df['data_rounds_ambiguity'])
    A = np.reshape(A,[70,1])
    SubjectChoices = np.array(df_temp['data_rounds_selected_variable_bet_reward'])
    SubjectChoices = np.reshape(SubjectChoices,[70,1])
    exog = np.concatenate((x,p,s,A), axis=1)
    endog = SubjectChoices

    return exog, endog   

def _ll_nb2(enog, exog, alpha, beta, gamma):

    x = exog[:,0]
    p = exog[:,1]
    s = exog[:,2]
    A = exog[:,3]
    SV_X = (p-beta*(A/2)) * np.power(x,alpha)
    SV_Y = np.power(s,alpha)
    prob = 1/(1+np.exp(-gamma*(SV_X-SV_Y)))
    temp = 1-prob
    prob[enog == 0] = temp[enog == 0]
    negLL = -np.log(prob)
    return negLL

class Bin(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(Bin, self).__init__(endog, exog, **kwds)

    def nloglikeobs(self,parms):
        alpha = parms[0]
        beta = parms[1]
        gamma = parms[2]
        negll = _ll_nb2(self.endog, self.exog, alpha, beta, gamma)          
        return negll

    def fit(self, start_params=None, method='L-BFGS-B',bounds = ((0, None), (-1, 1), (0, None)) ,maxiter=10000, maxfun=50000, **kwds):
        # we have one additional parameter and we need to add it for summary
        #self.exog_names.append('gamma')
        if start_params == None:
            # Reasonable starting values
            start_params = np.array([0.9,0,1])
            # intercept
            #start_params[-2] = np.log(self.endog.mean())
        return super(Bin, self).fit(start_params=start_params,method='lbfgs',bounds = ((0, None), (-1, 1), (0, None)),
                                     maxiter=maxiter, maxfun=maxfun,
                                     **kwds)
    
def RT(week,sub):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    subID = subID_list()
    week_label = [['01','01'],['08','11'],['17','19'],['23','26'],['32','34'],['36','42'],['44','48'],['52','56']]
    if (sub in subID99) == True:
        lottery_file_1 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_99_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][0]
        filelist_1 = zp.ZipFile(lottery_file_1).namelist()
        lottery_file_2 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_99_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][1]
        filelist_2 = zp.ZipFile(lottery_file_2).namelist()
    else:
        lottery_file_1 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_114_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][0]
        filelist_1 = zp.ZipFile(lottery_file_1).namelist()
        lottery_file_2 = '/Users/zyy219/Documents/Risk_project/Data/RP/study_114_Levy Probability Choice Task - Healthy North American_%s.zip'%week_label[week-1][1]
        filelist_2 = zp.ZipFile(lottery_file_2).namelist()

    sub_file = {}
    for i in range(len(subID)):
        key = subID[i]
        sub_file.setdefault(key, [])
        sub_file[key].extend([s for s in filelist_1 if subID[i] in s])
        sub_file[key].extend([s for s in filelist_2 if subID[i] in s])

    df1 = pd.read_csv(zp.ZipFile(lottery_file_1).open(sub_file[sub][0]))
    RT_start = df1['start_time'].to_list()
    RT_end = df1['end_time'].to_list()
    s = list(set(RT_start))
    e = list(set(RT_end))
    dt_start = datetime.strptime(s[0][0:19], "%Y-%m-%d %H:%M:%S").time()
    dt_end = datetime.strptime(e[0][0:19], "%Y-%m-%d %H:%M:%S").time()

    RT1 = datetime.combine(date.today(), dt_end) - datetime.combine(date.today(), dt_start)

    df2 = pd.read_csv(zp.ZipFile(lottery_file_2).open(sub_file[sub][1]))
    RT_start = df2['start_time'].to_list()
    RT_end = df2['end_time'].to_list()
    s = list(set(RT_start))
    e = list(set(RT_end))
    dt_start = datetime.strptime(s[0][0:19], "%Y-%m-%d %H:%M:%S").time()
    dt_end = datetime.strptime(e[0][0:19], "%Y-%m-%d %H:%M:%S").time()

    RT2 = datetime.combine(date.today(), dt_end) - datetime.combine(date.today(), dt_start)
    mean_RT = (RT1+RT2)/2
    return RT1, RT2

### Time -k, delay ratio, RT

In [8]:
def TD_ratio(week,sub):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    subID = subID_list()
    
    if (sub in subID99) == True:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_99_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    else:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_114_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    sub_file = {}
    for i in range(len(subID)):
        key = subID[i]
        sub_file.setdefault(key, [])
        sub_file[key].extend([s for s in filelist if subID[i] in s])
    
    file = sub_file[sub]
    
    if len(file) == 0:
        delay_ratio = 'NaN'
    else:
        df = pd.read_csv(zp.ZipFile(lottery_file).open(sub_file[sub][0]))
        delay = df[['data_rounds_delayed_reward', 'data_rounds_selected_immediate_reward']]
        delay_ratio = 1- delay.data_rounds_selected_immediate_reward.sum()/36
    #d3_ratio = round(delay_3.groupby(delay_3.data_rounds_delayed_reward).sum()/7,2)
    return delay_ratio

def TD_analysis_data(week,sub):
    
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    subID = subID_list()
    if (sub in subID99) == True:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_99_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    else:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_114_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    sub_file = {}
    for i in range(len(subID)):
        key = subID[i]
        sub_file.setdefault(key, [])
        sub_file[key].extend([s for s in filelist if subID[i] in s])
    
    file = sub_file[sub]
    
    if len(file) == 0:
        exog = 'NaN'
        endog = 'NaN'
    else:
        df = pd.read_csv(zp.ZipFile(lottery_file).open(sub_file[sub][0]))
        df.data_rounds_selected_immediate_reward = df.data_rounds_selected_immediate_reward.astype(int)
        df_temp = df[['participant_id','data_rounds_delay_days','data_rounds_delayed_reward','data_rounds_selected_immediate_reward']]

        df_temp.loc[:,'data_rounds_selected_immediate_reward'] = df_temp['data_rounds_selected_immediate_reward'].replace({0:1, 1:0})
        df_temp = df_temp.rename(columns={'data_rounds_selected_immediate_reward': "data_rounds_selected_delay_reward"})
        

        SubjectChoices = np.array(df_temp['data_rounds_selected_delay_reward'])
        SubjectChoices = np.reshape(SubjectChoices,[36,1])

        v = np.array(df_temp['data_rounds_delayed_reward'])
        v = np.reshape(v,[36,1])
        d = np.array(df_temp['data_rounds_delay_days'])
        d = np.reshape(d,[36,1])
        s = 5*np.ones((36,1))

        exog = np.concatenate((v,d,s), axis=1)
        endog = SubjectChoices
    return exog, endog
         

class TD_Bin(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(TD_Bin, self).__init__(endog, exog, **kwds)

    def nloglikeobs(self,parms):
        k = parms[0]
        gamma = parms[1]
        negll = TD_ll_nb2(self.endog, self.exog, k, gamma)          
        return negll

    def fit(self, start_params=None, method='lbfgs',bounds = ((0, 1), (0, None)) ,maxiter=10000, maxfun=50000, **kwds):
        # we have one additional parameter and we need to add it for summary
        #self.exog_names.append('gamma')
        if start_params == None:
            # Reasonable starting values
            start_params = np.array([0.5,1])
            # intercept
            #start_params[-2] = np.log(self.endog.mean())
        return super(TD_Bin, self).fit(start_params=start_params,method='lbfgs',bounds = ((0, 1), (0, None)),
                                     maxiter=maxiter, maxfun=maxfun,
                                     **kwds)
    
def TD_ll_nb2(endog, exog, k,gamma):
    #hyperbolic function
    v = exog[:,0]
    d = exog[:,1]
    s = exog[:,2]

    SV_X = v /(1+k*d) #delay
    SV_Y = s #now
    prob = 1/(1+np.exp(-gamma*(SV_X-SV_Y)))
    temp = 1-prob
    prob[endog == 0] = temp[endog == 0]
    negLL = -np.log(prob)
    return negLL

def TD_RT(week,sub):
    subID99 = ['797','806','809','826','843','844','845','855','856','857']
    subID = subID_list()
    
    if (sub in subID99) == True:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_99_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    else:
        lottery_file = '/Users/zyy219/Documents/Risk_project/Data/TD/study_114_Delayed Discounting - Healthy North American_week%s.zip'%week
        filelist = zp.ZipFile(lottery_file).namelist()
    sub_file = {}
    for i in range(len(subID)):
        key = subID[i]
        sub_file.setdefault(key, [])
        sub_file[key].extend([s for s in filelist if subID[i] in s])

    df = pd.read_csv(zp.ZipFile(lottery_file).open(sub_file[sub][0]))
    

    RT_start = df['start_time'].to_list()
    RT_end = df['end_time'].to_list()
    s = list(set(RT_start))
    e = list(set(RT_end))
    dt_start = datetime.strptime(s[0][0:19], "%Y-%m-%d %H:%M:%S").time()
    dt_end = datetime.strptime(e[0][0:19], "%Y-%m-%d %H:%M:%S").time()

    RT1 = datetime.combine(date.today(), dt_end) - datetime.combine(date.today(), dt_start)
    RT = RT1.total_seconds()
    
    return RT

In [9]:
Week = ['1','2','3','4','5','6','7','8']
subID = subID_list()
subNo = 49

all_k = np.empty([subNo, 8])
all_TD_gamma = np.empty([subNo, 8])
all_TD_RT = np.empty([subNo,8])
all_delay_ratio = np.empty([subNo, 8])

for i in range(len(subID)):
    for t in range(len(Week)):
        exog, endog = TD_analysis_data(Week[t],subID[i])
        if exog == "NaN":
            all_k[i][t] = float('NaN')
            all_TD_gamma[i][t] = float('NaN')
            all_TD_RT[i][t] = float('NaN')
            all_delay_ratio[i][t] = float('NaN')
            
        else:
            mod = TD_Bin(endog, exog)
            res = mod.fit()
            all_k[i][t] = res.params[0]
            all_TD_gamma[i][t] = res.params[1]
            all_TD_RT[i][t] = TD_RT(Week[t],subID[i])
            all_delay_ratio[i][t] = TD_ratio(Week[t], subID[i])

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  del sys.path[0]
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-pac

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

In [13]:
subID = subID_list()
week = [1,2,3,4,5,6,7,8]

subNo = 49

all_alpha = np.empty([subNo , 8])
all_beta = np.empty([subNo , 8])
all_gamma = np.empty([subNo, 8])
all_amb_ratio = np.empty([subNo, 8])
all_risk_ratio = np.empty([subNo, 8])
all_RT = np.empty([subNo, 8])

for i in range(len(subID)):
    for t in range(len(week)):
        amb_ratio, risk_ratio = rp_ratio(week[t],subID[i])
        #print(subID[i])
        df = raw_data(week[t],subID[i])
        exog, endog = analysis_data(df)
        mod = Bin(endog, exog)
        res = mod.fit()
        all_alpha[i][t] = res.params[0]
        all_beta[i][t] = res.params[1]
        all_gamma[i][t] = res.params[2]
        all_amb_ratio[i][t] =  amb_ratio
        all_risk_ratio[i][t] = risk_ratio
        RT1, RT2 = RT(week[t],subID[i])
        meanRT = (RT1.total_seconds()+RT2.total_seconds())/2
        all_RT[i][t] = meanRT

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in log
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_name

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWar

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWar

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWar

...

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWar

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWar

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more exog_names than parameters
  warnings.warn('more exog_names than parameters', ValueWarning)
//anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:919: ValueWarning: more 

In [17]:
all_risk_ratio

array([[0.41, 0.32, 0.26, 0.38, 0.47, 0.41, 0.38, 0.41],
       [0.53, 0.59, 0.32, 0.29, 0.26, 0.47, 0.35, 0.44],
       [0.41, 0.29, 0.44, 0.38, 0.41, 0.44, 0.5 , 0.53],
       [0.35, 0.24, 0.53, 0.68, 0.47, 0.5 , 0.53, 0.62],
       [0.82, 0.24, 0.24, 0.35, 0.32, 0.56, 0.41, 0.44],
       [0.47, 0.62, 0.53, 0.12, 0.21, 0.24, 0.21, 0.24],
       [0.65, 0.35, 0.26, 0.26, 0.18, 0.09, 0.03, 0.09],
       [0.76, 0.38, 0.24, 0.38, 0.32, 0.29, 0.29, 0.29],
       [0.88, 0.68, 0.82, 0.5 , 0.74, 0.76, 0.79, 0.85],
       [0.47, 0.91, 0.26, 0.41, 0.47, 0.53, 0.53, 0.47],
       [0.35, 0.18, 0.26, 0.35, 0.38, 0.32, 0.35, 0.29],
       [0.65, 0.38, 0.38, 0.38, 0.35, 0.35, 0.5 , 0.32],
       [1.  , 0.56, 0.62, 0.62, 0.82, 0.74, 0.82, 0.82],
       [0.29, 0.21, 0.26, 0.32, 0.18, 0.18, 0.18, 0.12],
       [0.35, 0.62, 0.71, 0.82, 0.85, 0.76, 0.76, 0.76],
       [0.35, 0.44, 0.38, 0.35, 0.38, 0.35, 0.24, 0.  ],
       [0.47, 0.38, 0.15, 0.26, 0.24, 0.18, 0.18, 0.24],
       [0.71, 0.62, 0.  , 0.  ,

### Emotion Score

In [12]:
def emotion_score(sub_id):
    positive = ['Happy', 'Enjoying myself', 'Warm/ friendly']
    negative =['4. Depressed/ blue', 'Angry / hostile', 'Criticized/put down \xa0\xa0\xa0', 'Worried/ anxious\xa0', 'Frustrated/annoyed', \
           'Hassled / pushed around']
      
    survey = ['DRM']
    subID = ['797','806','809','826','843','845','855','856','857']
    if (sub_id in subID) == True:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_99_participant_responses.csv'
    else:
        subfile = '/Users/zyy219/Documents/Risk_project/Data/Survey/study_114_participant_responses.csv'
    
    df = pd.read_csv(subfile)

    subsetDataFrame =  df[df['survey_title'].isin(survey) & df['participant_id'].isin([sub_id])]
    time = subsetDataFrame['start_by'].unique()
    
    p_score = []
    n_score = []

    for t in range(len(time)):
        df_temp = subsetDataFrame[subsetDataFrame['start_by'].isin([time[t]])]
        q_title =  df_temp['question_title'].to_list()
    
        positive_score = 0
        negative_score = 0
        answer_title = df_temp['answer_value'].to_list()
    
        for i in range(len(q_title)):
        
        
            
            if q_title[i] in positive:
                positive_score += int(answer_title[i][0]) 
            
            if q_title[i] in negative:
                negative_score += int(answer_title[i][0])   
        
        p_score.append(positive_score)
        n_score.append(negative_score)
        
    ps = np.round(np.array(p_score)/3,2)
    ns = np.round(np.array(n_score)/6,2)
    return ps, ns



In [13]:
subID = subID_list()
subNo = 49
### weekly emotion
positive_score = np.empty([subNo, 11])
negative_score = np.empty([subNo, 11])

for i in range(len(subID)):
    ps, ns = emotion_score(subID[i])
    #print(subID[i])
    if subID[i] == '1262':
        ps = np.delete(ps, [1])
        ns = np.delete(ns, [1])
    positive_score[i][:] = ps
    negative_score[i][:] = ns

#Week = [['1'],['8','11'],['17'],['23'],['32'],['36'],['44','48'],['52','56']]
#positive score 
w1 = positive_score[:,0]
w2 = np.sum(positive_score[:,1:3],axis = 1)/2
w7 = np.sum(positive_score[:,7:9],axis = 1)/2
w8 = np.sum(positive_score[:,9:11],axis = 1)/2

positive = np.empty([subNo,8])

positive[:,0] = w1
positive[:,1] = w2
positive[:,2:6] = positive_score[:,2:6]
positive[:,6] = w7
positive[:,7] = w8

#negative score
nw1 = negative_score[:,0]
nw2 = np.sum(negative_score[:,1:3],axis = 1)/2
nw7 = np.sum(negative_score[:,7:9],axis = 1)/2
nw8 = np.sum(negative_score[:,9:11],axis = 1)/2

negative = np.empty([subNo,8])

negative[:,0] = nw1
negative[:,1] = nw2
negative[:,2:6] = negative_score[:,2:6]
negative[:,6] = nw7
negative[:,7] = nw8

### MEAN PARAMETER I & II 

In [15]:
subNO = 49
weekNO = 8

## Parameter I 
mean_alpha = np.sum(all_alpha,1).reshape((subNO, 1))/weekNO
mean_beta = np.sum(all_beta,1).reshape((subNO, 1))/weekNO
mean_risk_ratio = np.sum(all_risk_ratio,1).reshape((subNO, 1))/weekNO
mean_amb_ratio = np.sum(all_amb_ratio,1).reshape((subNO, 1))/weekNO
mean_k = np.empty([subNO,1])
mean_d = np.empty([subNO,1])

## Parameter II 
mean_gamma = np.sum(all_gamma,1).reshape((subNO, 1))/weekNO
mean_RT = np.sum(all_RT,1).reshape((subNO, 1))/weekNO
mean_positive = np.sum(positive,1).reshape((subNO,1))/weekNO
mean_negative = np.sum(negative,1).reshape((subNO,1))/weekNO
mean_TD_gamma = np.empty([subNO,1])
mean_TD_RT = np.empty([subNO,1])

for i in range(subNO):
    p = np.argwhere(np.isnan(all_k[i][:]))
    if len(p) != 0:
        k = np.delete(all_k[i][:],p)
        d = np.delete(all_delay_ratio[i][:],p)
        mk = np.sum(k)/len(k)
        md = np.sum(d)/len(d)
        mean_k[i] = mk
        mean_d[i] = md 
        rt = np.delete(all_TD_RT[i][:],p)
        gamma = np.delete(all_TD_gamma[i][:],p)
        mrt = np.sum(rt)/len(rt)
        mg = np.sum(gamma)/len(gamma)
        mean_TD_RT[i] = mrt
        mean_gamma[i] = mg
    else:
        k = all_k[i][:]
        d = all_delay_ratio[i][:]
        mk = np.sum(k)/len(k)
        md = np.sum(d)/len(d)
        mean_k[i] = mk
        mean_d[i] = md
        rt = all_TD_RT[i][:]
        gamma = all_TD_gamma[i][:]
        mrt = np.sum(rt)/len(rt)
        mg = np.sum(gamma)/len(gamma)
        mean_TD_RT[i] = mrt
        mean_TD_gamma[i] = mg
        
para_mtrx = np.array([mean_alpha,mean_risk_ratio,mean_beta,mean_amb_ratio,mean_k,mean_d]).reshape(6,subNO)
para_mtrx_II = np.array([mean_gamma,mean_RT,mean_TD_gamma,mean_TD_RT,mean_positive,mean_negative]).reshape(6,subNO)


### Uncertainty Scale - DOSPERT, IUS, LOT

In [29]:
subID = subID_list()
trait = np.empty([subNO,9])
for i in range(len(subID)):
    dosert_label, dosert_score = dosert(subID[i])
    ius_label, ius_score = IUS_score(subID[i])
    lot_label, lot_score = lot(subID[i])
    
    trait[i][0:5] = np.array(dosert_score)
    trait[i][5:7] = np.array(ius_score)
    trait[i][7:9] = np.array(lot_score)
    
corr_matx = np.empty([6,9])
corr_matxII = np.empty([6,9])
corr_p = np.empty([6,9])
corr_p_II = np.empty([6,9])

for i in range(6):
    data1 = para_mtrx[i][:].tolist()
    data3 = para_mtrx_II[i][:].tolist()
    for t in range(9): 
        data2 = trait[:,t].tolist()
        corr, _ = pearsonr(data1, data2)
        corr3,_3 = pearsonr(data3,data2)
        
        corr_matx[i][t] = corr
        corr_matxII[i][t] = corr3
        corr_p[i][t] = _    
        corr_p_II[i][t] = _3
        
cor_matx = np.round(corr_matx,4) 
cor_p_unc = np.round(corr_p,4)
cor_matxII = np.round(corr_matxII,4)
cor_p_unc_II = np.round(corr_p_II,3)

In [23]:
cor_p_unc

array([[0.5833, 0.4501, 0.1444, 0.9162, 0.4166, 0.7486, 0.847 , 0.1411,
        0.9889],
       [0.6731, 0.0357, 0.0027, 0.478 , 0.7764, 0.3942, 0.1814, 0.1961,
        0.4569],
       [0.2494, 0.756 , 0.3845, 0.3891, 0.9238, 0.3971, 0.5772, 0.1136,
        0.9023],
       [0.3286, 0.3126, 0.3655, 0.9049, 0.598 , 0.2729, 0.2872, 0.5957,
        0.8774],
       [0.3417, 0.7682, 0.3935, 0.3987, 0.1877, 0.0251, 0.0159, 0.4472,
        0.803 ],
       [0.449 , 0.7919, 0.8147, 0.2101, 0.5655, 0.049 , 0.1373, 0.0953,
        0.7958]])

### Impulsivity

In [34]:
trait_imp = np.empty([subNO,4])
for i in range(len(subID)):
    basbis_label, basbis_score = bisbas(subID[i])
    #print(basbis_label, basbis_score)
    bis11_score = bis11(subID[i])
    upps_score = UPPS(subID[i])
    trait_imp[i][0:2] = np.array(basbis_score)
    trait_imp[i][2] = np.array(bis11_score)
    trait_imp[i][3] = np.array(upps_score)
    
corr_matx = np.empty([6,4])
corr_matxII = np.empty([6,4])
corr_p = np.empty([6,4])
corr_p_II = np.empty([6,4])

for i in range(6):
    data1 = para_mtrx[i][:].tolist()
    data3 = para_mtrx_II[i][:].tolist()
    for t in range(4): 
        data2 = trait_imp[:,t].tolist()
        corr, _ = pearsonr(data1, data2)
        corr3,_3 = pearsonr(data3,data2)
        
        corr_matx[i][t] = corr
        corr_matxII[i][t] = corr3
        corr_p[i][t] = _    
        corr_p_II[i][t] = _3 
        
cor_matx_imp = np.round(corr_matx,4)
cor_p_imp = np.round(corr_p,4)   
cor_matx_imp_II = np.round(corr_matxII,4)
cor_p_imp_II = np.round(corr_p_II,3)  

In [37]:
cor_p_imp_II

array([[0.862, 0.22 , 0.67 , 0.121],
       [0.708, 0.807, 0.093, 0.869],
       [0.047, 0.479, 0.833, 0.822],
       [0.487, 0.958, 0.237, 0.975],
       [0.156, 0.468, 0.195, 0.001],
       [0.533, 0.088, 0.001, 0.   ]])

In [86]:
BSI_score = BSI('797')
np.sum(BSI_score)

45

### Mental Health

In [87]:
mental_variable_No = 5
trait_mental = np.empty([subNO,mental_variable_No])
for i in range(len(subID)):
    phq_score = phq(subID[i])
    stais_score = stai_s(subID[i])
    stait_score = stai_t(subID[i])
    BAI_score = BAI(subID[i])
    BSI_score = BSI(subID[i])
    trait_mental[i][0] = np.array(phq_score)
    trait_mental[i][1] = np.array(stais_score)
    trait_mental[i][2] = np.array(stait_score)
    trait_mental[i][3] = np.array(BAI_score)
    #trait_mental[i][4:mental_variable_No] = np.array(BSI_score)
    trait_mental[i][4] = np.array(np.sum(BSI_score))
    
#print(trait_mental)    
corr_matx = np.empty([6,mental_variable_No])
corr_matxII = np.empty([6,mental_variable_No])
corr_p = np.empty([6,mental_variable_No])
corr_p_II = np.empty([6,mental_variable_No])


for i in range(6):
    data1 = para_mtrx[i][:].tolist()
    data3 = para_mtrx_II[i][:].tolist()
    for t in range(mental_variable_No): 
        data2 = trait_mental[:,t].tolist()
        corr, _ = pearsonr(data1, data2)
        corr3,_3 = pearsonr(data3,data2)
        corr_matx[i][t] = corr
        corr_matxII[i][t] = corr3
        corr_p[i][t] = _  
        corr_p_II[i][t] = _3 
cor_matx_mental = np.round(corr_matx,3)
cor_matx_mental_II = np.round(corr_matxII,3)
cor_p_mental = np.round(corr_p,3)  
cor_p_mental_II = np.round(corr_p_II,3)  

In [114]:
cor_matx_mental

array([[ 0.112,  0.015,  0.055, -0.05 ,  0.235],
       [ 0.282,  0.084,  0.196,  0.151,  0.41 ],
       [ 0.095, -0.053, -0.039,  0.077, -0.045],
       [ 0.123,  0.09 ,  0.14 ,  0.034,  0.279],
       [-0.141, -0.144, -0.122, -0.071, -0.046],
       [ 0.1  ,  0.121,  0.168,  0.036,  0.025]])

In [116]:
cor_p_mental_II

array([[0.367, 0.638, 0.877, 0.516, 0.505],
       [0.903, 0.051, 0.858, 0.929, 0.124],
       [0.56 , 0.163, 0.401, 0.08 , 0.981],
       [0.286, 0.062, 0.921, 0.354, 0.398],
       [0.017, 0.002, 0.   , 0.036, 0.107],
       [0.001, 0.   , 0.   , 0.   , 0.   ]])

### Cognition

In [80]:
cognition_variable_No = 2
trait_cognition = np.empty([subNO,cognition_variable_No])
for i in range(len(subID)):
    IQ_score = Raven_score(subID[i])
    numeracy_score = numeracy(subID[i])
    trait_cognition[i,0] = np.array(IQ_score)
    trait_cognition[i,1] = np.array(numeracy_score)
    
pos = np.argwhere(np.isnan(trait_cognition))

trait_cognition


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

array([[60.,  5.],
       [55.,  5.],
       [55.,  5.],
       [54.,  5.],
       [47.,  2.],
       [37.,  5.],
       [43.,  4.],
       [33.,  3.],
       [47.,  4.],
       [13.,  2.],
       [55.,  4.],
       [nan, nan],
       [45.,  3.],
       [54.,  3.],
       [55.,  6.],
       [53.,  4.],
       [52.,  3.],
       [48.,  2.],
       [42.,  3.],
       [51.,  3.],
       [36.,  3.],
       [31.,  3.],
       [40.,  2.],
       [20.,  6.],
       [50.,  3.],
       [nan, nan],
       [nan,  3.],
       [55.,  5.],
       [27.,  3.],
       [53.,  3.],
       [45.,  2.],
       [22.,  1.],
       [56.,  5.],
       [39.,  5.],
       [39.,  2.],
       [57.,  5.],
       [56.,  3.],
       [53.,  2.],
       [59.,  6.],
       [19.,  1.],
       [38.,  2.],
       [58.,  3.],
       [35.,  3.],
       [54.,  2.],
       [43.,  3.],
       [48.,  5.],
       [54.,  4.],
       [52.,  1.],
       [43.,  5.]])

In [ ]:
for i in range(6):
    data1 = np.delete(para_mtrx[i][:],pos).tolist()
    data3 = np.delete(para_mtrx_II[i][:],pos).tolist()
    data2 = np.delete(trait_cognition,pos).tolist()
    data4 = np.delete(numeracy_score,pos_math
    corr, _ = pearsonr(data1, data2)
    print(corr)
    #print(_)
    corr3,_3 = pearsonr(data3,data2)
    #print(corr3)

### DOSPERT correlation- financial, safety

In [32]:
for i in range(15):
    data1 = trait[:,1]
    data2 = trait[:,2]
    data3 = trait_mental[:,i]
    #corr,_ = pearsonr(data1,data2)
    corr,_ = pearsonr(data2, data3)
    
    #print(corr)
    print(_)

0.03075843871795886
0.4895257270650203
0.006846748419273341
0.10847301307035069
0.35453510168693436
0.0014313020118077511
0.0219426721086826
0.013585272666930288
0.11144022563686633
0.17014754608976643
0.07879565948900424
0.06582772240295008
0.013239472673611292
0.04357079267558465
0.8459364532818922


In [ ]:
for i in range(12): 
    data1 = np.mean(all_RT, axis = 1)
    data2 = trait_mental[:,i]
    corr,_ = pearsonr(data1,data2)
    #print(corr)
    print(_)

In [36]:
corr_matx = np.empty([12,49])
corr_matx[0][:] = np.mean(all_alpha,axis = 1) #all_risk_ratio
corr_matx[1][:] = np.mean(all_risk_ratio, axis = 1)
corr_matx[2][:] = np.mean(all_beta, axis = 1)
corr_matx[3][:] = np.mean(all_amb_ratio, axis = 1)
corr_matx[4][:] = np.mean(all_gamma, axis = 1)
corr_matx[5][:] = np.mean(all_RT, axis = 1)

mean_TD_RT = np.empty([49])
mean_k = np.empty([49])
mean_TD_gamma = np.empty([49])
mean_delay_ratio = np.empty([49])

for i in range(49):
    x = all_TD_RT[i,:]
    x = x[~np.isnan(x)]
    x1 = all_k[i,:]
    x1 = x1[~np.isnan(x1)]
    x2 = all_delay_ratio[i,:]
    x2 = x2[~np.isnan(x2)]
    x3 = all_TD_gamma[i,:]
    x3 = x3[~np.isnan(x3)]
    mean_TD_RT[i] = np.mean(x)
    mean_k[i] = np.mean(x1)
    mean_delay_ratio[i] = np.mean(x2)
    mean_TD_gamma[i] = np.mean(x3)
    
corr_matx[6][:] = mean_k
corr_matx[7][:] = mean_delay_ratio
corr_matx[8][:] = mean_TD_gamma
corr_matx[9][:] = mean_TD_RT
corr_matx[10][:] = np.mean(positive, axis = 1)
corr_matx[11][:] = np.mean(negative, axis = 1)

In [37]:
corr_all = np.empty([12,12])
corr_all_p = np.empty([12,12])
for i in range(12):
    data1 = corr_matx[i][:]
    for t in range(12):
        data2 = corr_matx[t][:]
        corr, _ = pearsonr(data1,data2)
        corr_all[i][t] = corr
        corr_all_p[i][t] = _
np.round(corr_all,2)
np.round(corr_all_p,2)

array([[0.  , 0.  , 0.44, 0.  , 0.34, 0.38, 0.06, 0.17, 0.71, 0.97, 0.45,
        0.21],
       [0.  , 0.  , 0.23, 0.  , 0.05, 0.9 , 0.13, 0.56, 0.85, 0.19, 0.66,
        0.28],
       [0.44, 0.23, 0.  , 0.  , 0.3 , 0.33, 0.02, 0.01, 0.42, 0.05, 0.63,
        0.19],
       [0.  , 0.  , 0.  , 0.  , 0.02, 0.57, 0.01, 0.05, 0.6 , 0.33, 0.89,
        0.1 ],
       [0.34, 0.05, 0.3 , 0.02, 0.  , 0.84, 0.69, 0.38, 0.32, 0.21, 0.12,
        0.7 ],
       [0.38, 0.9 , 0.33, 0.57, 0.84, 0.  , 0.06, 0.07, 0.27, 0.  , 1.  ,
        0.03],
       [0.06, 0.13, 0.02, 0.01, 0.69, 0.06, 0.  , 0.  , 0.08, 0.1 , 0.8 ,
        0.62],
       [0.17, 0.56, 0.01, 0.05, 0.38, 0.07, 0.  , 0.  , 0.01, 0.03, 0.51,
        0.53],
       [0.71, 0.85, 0.42, 0.6 , 0.32, 0.27, 0.08, 0.01, 0.  , 0.9 , 0.02,
        0.36],
       [0.97, 0.19, 0.05, 0.33, 0.21, 0.  , 0.1 , 0.03, 0.9 , 0.  , 0.97,
        0.28],
       [0.45, 0.66, 0.63, 0.89, 0.12, 1.  , 0.8 , 0.51, 0.02, 0.97, 0.  ,
        0.01],
       [0.21, 0.28, 0

In [38]:
d1 = np.mean(all_risk_ratio, axis = 1)
d0 = np.mean(all_RT, axis = 1)
d2 = np.mean(negative, axis = 1)

print(d1)


corr,_ = pearsonr(d1,d2)
print(corr)

print(_)


[0.38    0.40625 0.425   0.49    0.4225  0.33    0.23875 0.36875 0.7525
 0.50625 0.31    0.41375 0.75    0.2175  0.70375 0.31125 0.2625  0.16625
 0.53375 0.37875 0.18    0.04875 0.795   0.04125 0.5125  0.67625 0.39125
 0.54125 0.8925  0.02625 0.38    0.35    0.1725  0.      0.43    0.25375
 0.61625 0.35    0.04125 0.06    0.4475  0.4775  0.455   0.49625 0.78875
 0.2125  0.16625 0.1925  0.28125]
0.15873674285953998
0.2759786188868782


In [77]:
corr_matx = np.empty([15,15])
for i in range(15):
        data1 = trait_mental[:,i]
        
        for t in range(15): 
            data2 = trait_mental[:,t]
            
            corr, _ = pearsonr(data1, data2)
            #print(corr)
            corr_matx[t][i] = corr

In [ ]:
mean_TD_RT = np.empty([49])
for i in range(39):
    x = all_TD_RT[i,:]
    x = x[~np.isnan(x)]
    mean_TD_RT[i] = np.mean(x)


### regression 

In [67]:
y = np.mean(all_risk_ratio, axis = 1)
y_RT = np.mean(all_RT, axis = 1)

#this was based on the correlation matrix
global_score_array = np.concatenate((trait_mental[:,4:8],trait_mental[:,9:13]),axis = 1)
global_score = np.sum(global_score_array,axis=1)


In [61]:
bsi_global = np.sum(trait_mental[:,4:], axis = 1)
ius_global = np.sum(trait[:,5:7], axis = 1) 


### full model regression

In [118]:
x = np.empty([49,13])
x[:,0:5] = trait[:,0:5]
x[:,5] = ius_global
x[:,6:8] = trait[:,7:]
x[:,8:12] = trait_imp
x[:,12] = trait_mental[:,0]


exog = sm.add_constant(x, prepend=False)
mod = sm.OLS(y, exog)
res = mod.fit()
print(res.summary())

#for RT



                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     3.048
Date:                Wed, 26 Aug 2020   Prob (F-statistic):            0.00430
Time:                        14:11:32   Log-Likelihood:                 23.845
No. Observations:                  49   AIC:                            -19.69
Df Residuals:                      35   BIC:                             6.796
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0103      0.009     -1.127      0.2

# full model regression, just one mental score

In [72]:
mental_global = np.sum(trait_mental, axis = 1)
#mental_mean = np.mean(trait_mental, axis = 1)
x = np.empty([49,13])
x[:,0:5] = trait[:,0:5]
x[:,5] = ius_global
x[:,6:8] = trait[:,7:]
x[:,8:12] = trait_imp
x[:,12] = mental_global
#x[:,12] = mental_mean
exog = sm.add_constant(x, prepend=False)
mod = sm.OLS(y, exog)
res = mod.fit()
print(res.summary())

#mod = sm.OLS(y_RT, exog)
#res = mod.fit()
#print(res.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     2.772
Date:                Tue, 25 Aug 2020   Prob (F-statistic):            0.00818
Time:                        11:21:57   Log-Likelihood:                 22.634
No. Observations:                  49   AIC:                            -17.27
Df Residuals:                      35   BIC:                             9.218
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0138      0.009     -1.509      0.1

In [76]:
mental_global = np.sum(trait_mental, axis = 1)
#mental_mean = np.mean(trait_mental, axis = 1)
x = np.empty([49,4])
x[:,0:2] = trait[:,1:3]
x[:,2] = trait[:,7]
#x[:,3] = bsi_global
x[:,3] = mental_global

exog = sm.add_constant(x, prepend=False)
mod = sm.OLS(y, exog)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     5.845
Date:                Tue, 25 Aug 2020   Prob (F-statistic):           0.000732
Time:                        11:39:36   Log-Likelihood:                 15.734
No. Observations:                  49   AIC:                            -21.47
Df Residuals:                      44   BIC:                            -12.01
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0080      0.005      1.551      0.1

### model specified by the correlation matrix

In [95]:
x = np.empty([49,3])
x[:,0:2] = trait[:,1:3]
#x[:,2] = trait[:,7]
#x[:,3] = bsi_global
x[:,2] = trait_mental[:,0]
#x[:,2] = bsi_global

exog = sm.add_constant(x, prepend=False)
mod = sm.OLS(y, exog)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.275
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     5.702
Date:                Wed, 26 Aug 2020   Prob (F-statistic):            0.00214
Time:                        12:26:33   Log-Likelihood:                 13.187
No. Observations:                  49   AIC:                            -18.37
Df Residuals:                      45   BIC:                            -10.81
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0118      0.006      2.142      0.0

### the regression model for RTs

In [113]:
y_RT = np.mean(all_RT, axis = 1)
x = np.empty([49,1])
#x[:,0] = trait[:,8]
#x[:,2] = trait[:,7]
#x[:,1] = bsi_global
x[:,0] = trait_mental[:,1]

exog = sm.add_constant(x, prepend=False)
mod = sm.OLS(y_RT, exog)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     4.014
Date:                Wed, 26 Aug 2020   Prob (F-statistic):             0.0509
Time:                        12:44:49   Log-Likelihood:                -268.72
No. Observations:                  49   AIC:                             541.4
Df Residuals:                      47   BIC:                             545.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -1.9044      0.951     -2.004      0.0

### correlation between impulsivity 

In [123]:
corrmatx = np.empty([4,4])

for i in range(4):
    data1 = trait_imp[:,i]
    for t in range(4):
        data2 = trait_imp[:,t]
        corr, _ = pearsonr(data1, data2)
        corrmatx[i][t] = corr
        #print(corr)

In [124]:
corrmatx

array([[ 1.        ,  0.35320355,  0.05948759, -0.18087417],
       [ 0.35320355,  1.        , -0.17408552, -0.13166735],
       [ 0.05948759, -0.17408552,  1.        ,  0.17728084],
       [-0.18087417, -0.13166735,  0.17728084,  1.        ]])

### correlation between uncertainty

In [130]:
corrmatx = np.empty([9,9])

for i in range(9):
    data1 = trait[:,i]
    for t in range(9):
        data2 = trait[:,t]
        corr, _ = pearsonr(data1, data2)
        corrmatx[i][t] = corr
        #print(corr)

In [131]:
np.round(corrmatx,2)

array([[ 1.  ,  0.51,  0.43,  0.31,  0.44,  0.29,  0.33, -0.2 ,  0.2 ],
       [ 0.51,  1.  ,  0.2 ,  0.47,  0.24, -0.02, -0.11,  0.12,  0.12],
       [ 0.43,  0.2 ,  1.  ,  0.35,  0.51,  0.22,  0.36, -0.08,  0.1 ],
       [ 0.31,  0.47,  0.35,  1.  ,  0.45, -0.1 , -0.16,  0.21,  0.03],
       [ 0.44,  0.24,  0.51,  0.45,  1.  ,  0.02,  0.14,  0.21, -0.08],
       [ 0.29, -0.02,  0.22, -0.1 ,  0.02,  1.  ,  0.74, -0.44,  0.49],
       [ 0.33, -0.11,  0.36, -0.16,  0.14,  0.74,  1.  , -0.29,  0.37],
       [-0.2 ,  0.12, -0.08,  0.21,  0.21, -0.44, -0.29,  1.  , -0.48],
       [ 0.2 ,  0.12,  0.1 ,  0.03, -0.08,  0.49,  0.37, -0.48,  1.  ]])

In [137]:

data2 = 

In [140]:
data1 = trait[:,-1]
for i in range(5):
    data2 = trait_mental[:,i]
    corr, _ = pearsonr(data1, data2)
    print(corr)
    print(_)

0.357185906713837
0.011752974664775635
0.6339906993230419
1.0059681019763833e-06
0.6217191350707332
1.861822550875051e-06
0.3126097613897787
0.028749010883251186
0.5820125632240514
1.1521781330938812e-05
